<h1>410821322 Steven Jonathan</h1>

<h2>Setting up the data</h2>

<h3>Removing excessive lines at the end of horse-colic.data</h3>

In [1]:
line = open('horse-colic.data', 'r', encoding='utf-8').readlines() #Remove the endspace so that fitting won't be a problem
# line = ' '.join(line.split())
temp = []
for i in line:
    i = i.strip()
    temp.append(i+'\n')
# print(temp)
line = open('horse-colic.data', 'w', encoding='utf-8')
line.writelines(temp)
line.close()

<h3>Removing excessive lines at the end of horse-colic.test</h3>

In [2]:
line = open('horse-colic.test', 'r', encoding='utf-8').readlines() #Remove the endspace so that fitting won't be a problem
# line = ' '.join(line.split())
temp = []
for i in line:
    i = i.strip()
    temp.append(i+'\n')
# print(temp)
line = open('horse-colic.test', 'w', encoding='utf-8')
line.writelines(temp)
line.close()

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

#Setting up data
headernames = ["Surgery?","Age","HospitalNo","RectalTemp","Pulse","RespiratoryRate","ExtremitiesTemp","PeripheralPulse","MucousMembranes","CapillaryRefillTime","PainLevel","Peristalsis","AbdominalDistension","NasogastricTube","NasogastricReflux","NasogastricRefluxPH","Feces","Abdomen","PackedCellVolume","TotalProtein","AbdominocentesisAppearance","AbdominocentesisTotalProtein","Outcome","SurgicalLesion","LesionTypeOne","LesionTypeTwo","LesionTypeThree","CPData"]

<h3>Get the data for X,Y train and also replace NaN with high number</h3>

In [4]:
horsedataoriginal = pd.read_csv("horse-colic.data",sep=" ",index_col=False,names=headernames).replace('?', 999) #change the data file path, also replace ? with NaN for easier process
horsedataoriginal = horsedataoriginal.iloc[:,:24]

horsedataoriginal = horsedataoriginal.drop(columns=["HospitalNo"]) #Remove unused factors
horsedataoriginal = horsedataoriginal[0:]
horsedataoriginal["Outcome"] = horsedataoriginal["Outcome"].replace("3","2")
horsedataoriginal
# display(HTML(horsedataoriginal.to_html())) #To display all data

,Surgery?,Age,RectalTemp,Pulse,RespiratoryRate,ExtremitiesTemp,PeripheralPulse,MucousMembranes,CapillaryRefillTime,PainLevel,...,NasogastricReflux,NasogastricRefluxPH,Feces,Abdomen,PackedCellVolume,TotalProtein,AbdominocentesisAppearance,AbdominocentesisTotalProtein,Outcome,SurgicalLesion
0,2,1,38.50,66,28,3,3,999,2,5,...,999,999,3,5,45.00,8.40,999,999,2,2
1,1,1,39.2,88,20,999,999,4,1,3,...,999,999,4,2,50,85,2,2,2,2
2,2,1,38.30,40,24,1,1,3,1,3,...,999,999,1,1,33.00,6.70,999,999,1,2
3,1,9,39.10,164,84,4,1,6,2,2,...,2,5.00,3,999,48.00,7.20,3,5.30,2,1
4,2,1,37.30,104,35,999,999,6,2,999,...,999,999,999,999,74.00,7.40,999,999,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1,1,999,120,70,4,999,4,2,2,...,999,999,999,5,55,65,999,999,2,2
296,2,1,37.20,72,24,3,2,4,2,4,...,1,999,4,4,44.00,999,3,3.30,2,1
297,1,1,37.50,72,30,4,3,4,1,4,...,1,999,3,5,60.00,6.80,999,999,2,1
298,1,1,36.50,100,24,3,3,3,1,3,...,1,999,4,4,50.00,6.00,3,3.40,1,1


In [5]:
horsedata = horsedataoriginal
x_train = horsedata.loc[:,horsedata.columns!="Outcome"].astype(float)
y_train = horsedata["Outcome"].astype(float)

<h3>Get the data for X,Y test and also replace NaN with high number</h3>

In [6]:
horsedatatest = pd.read_csv("horse-colic.test",sep=" ",index_col=False,names=headernames).replace('?', 999)#change the data file path, also replace ? with NaN for easier process
horsedatatest = horsedatatest.iloc[:,:24]
horsedatatest = horsedatatest.drop(columns=["HospitalNo"]) #Remove unused factors
horsedatatest = horsedatatest[0:]
horsedatatest["Outcome"] = horsedatatest["Outcome"].replace("3","2")
horsedatatest

,Surgery?,Age,RectalTemp,Pulse,RespiratoryRate,ExtremitiesTemp,PeripheralPulse,MucousMembranes,CapillaryRefillTime,PainLevel,...,NasogastricReflux,NasogastricRefluxPH,Feces,Abdomen,PackedCellVolume,TotalProtein,AbdominocentesisAppearance,AbdominocentesisTotalProtein,Outcome,SurgicalLesion
0,2,1,38.50,54,20,999,1,2,2,3,...,2,5.90,999,2,42.00,6.30,999,999,1,2
1,2,1,37.60,48,36,999,999,1,1,999,...,999,999,999,999,44.00,6.30,1,5.00,1,2
2,1,1,37.7,44,28,999,4,3,2,5,...,1,999,3,5,45,70,3,2,1,1
3,1,1,37,56,24,3,1,4,2,4,...,1,999,999,999,35,61,3,2,2,2
4,2,1,38.00,42,12,3,999,3,1,1,...,999,999,999,2,37.00,5.80,999,999,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,2,1,38.60,60,30,1,1,3,1,4,...,1,999,999,999,40.00,6.00,1,999,1,1
64,2,1,37.80,42,40,1,1,1,1,1,...,999,999,3,3,36.00,6.20,999,999,1,2
65,1,1,38,60,12,1,1,2,1,2,...,1,999,1,4,44,65,3,2,2,1
66,2,1,38.00,42,12,3,999,3,1,1,...,999,999,999,1,37.00,5.80,999,999,1,2


In [7]:
horsedata = horsedatatest
x_test = horsedata.loc[:,horsedata.columns!="Outcome"].astype(float)
y_test = horsedata["Outcome"].astype(float)

<h2>Creating the KNeighborsClassifier</h2>

In [37]:
Kclassifier = KNeighborsClassifier(n_neighbors=43,metric='euclidean')
Kclassifier.fit(x_train,y_train)
y_pred = Kclassifier.predict(x_test)
print("Accuracy: ",round(accuracy_score(y_test, y_pred)*100,2),"%")

Accuracy:  73.53 %


<h2>Creating the KNNImputer</h2>


In [43]:
Kclassifier = KNNImputer(n_neighbors=43)
y_pred = Kclassifier.fit_transform(x_train,y_train)
y_pred
# y_pred = Kclassifier.predict(x_test)
# print("Accuracy: ",round(accuracy_score(y_test, y_pred)*100,2),"%")

array([[  2. ,   1. ,  38.5, ..., 999. , 999. ,   2. ],
       [  1. ,   1. ,  39.2, ...,   2. ,   2. ,   2. ],
       [  2. ,   1. ,  38.3, ..., 999. , 999. ,   2. ],
       ...,
       [  1. ,   1. ,  37.5, ..., 999. , 999. ,   1. ],
       [  1. ,   1. ,  36.5, ...,   3. ,   3.4,   1. ],
       [  1. ,   1. ,  37.2, ...,   1. ,   1. ,   2. ]])